In [1]:
import pandas as pd
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">
<h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Step 1: Attribute Selection</h1>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Import Data</h1><br><br> 


<p style="line-height:31px;">First let us read the contents of the file ad-clicks.csv. The following commands read in the CSV file in a table format and removes any extra whitespaces. So, if the CSV contained ' userid  ' it becomes 'userid'. <br><br>


Note that you must change the path to ad-clicks.csv to the location on your machine, if you want to run this command on your machine.
</p>

</div>

In [9]:
adclicksDF = pd.read_csv('../data/flamingo-data/ad-clicks.csv')
adclicksDF = adclicksDF.rename(columns=lambda x: x.strip()) #remove whitespaces from headers
adclicksDF.describe()

,txId,userSessionId,teamId,userId,adId
count,16323.000000,16323.000000,16323.000000,16323.000000,16323.000000
mean,24613.829259,22090.773449,70.294921,1187.464192,14.654046
std,9513.244787,8780.273065,39.631995,691.561945,8.623599
min,5972.000000,5649.000000,2.000000,1.000000,0.000000
25%,16994.500000,15880.000000,35.000000,564.000000,7.000000
50%,25111.000000,21017.000000,69.000000,1161.000000,15.000000
75%,32597.500000,27912.000000,99.000000,1771.000000,22.000000
max,39833.000000,39623.000000,179.000000,2387.000000,29.000000


In [10]:
adclicksDF.head(n=5)

,timestamp,txId,userSessionId,teamId,userId,adId,adCategory
0,2016-05-26 15:13:22,5974,5809,27,611,2,electronics
1,2016-05-26 15:17:24,5976,5705,18,1874,21,movies
2,2016-05-26 15:22:52,5978,5791,53,2139,25,computers
3,2016-05-26 15:22:57,5973,5756,63,212,10,fashion
4,2016-05-26 15:22:58,5980,5920,9,1027,20,clothing



<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Next, We are going to add an extra column to the ad-clicks table and make it equal to 1. We do so to record the fact that each ROW is 1 ad-click. 
You will see how this will become useful when we sum up this column to find how many ads
did a user click.</p>

</div>


In [11]:
adclicksDF['adCount'] = 1


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Let us display the first 5 lines of adclicksDF and see if 
a new column has been added:</p>

</div>

In [12]:
adclicksDF.head(n=5)

,timestamp,txId,userSessionId,teamId,userId,adId,adCategory,adCount
0,2016-05-26 15:13:22,5974,5809,27,611,2,electronics,1
1,2016-05-26 15:17:24,5976,5705,18,1874,21,movies,1
2,2016-05-26 15:22:52,5978,5791,53,2139,25,computers,1
3,2016-05-26 15:22:57,5973,5756,63,212,10,fashion,1
4,2016-05-26 15:22:58,5980,5920,9,1027,20,clothing,1


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Next, let us read the contents of the file buy-clicks.csv. As before, the following commands read in the CSV file in a table format and removes any extra whitespaces. So, if the CSV contained ' userid  ' it becomes 'userid'. <br><br>


Note that you must change the path to buy-clicks.csv to the location on your machine, if you want to run this command on your machine.
</p>

</div>

In [14]:
buyclicksDF = pd.read_csv('../data/flamingo-data/buy-clicks.csv')
buyclicksDF = buyclicksDF.rename(columns=lambda x: x.strip()) #removes whitespaces from headers



<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Let us display the first 5 lines of buyclicksDF:</p>

</div>

In [15]:
buyclicksDF.head(n=5)

,timestamp,txId,userSessionId,team,userId,buyId,price
0,2016-05-26 15:36:54,6004,5820,9,1300,2,3.0
1,2016-05-26 15:36:54,6005,5775,35,868,4,10.0
2,2016-05-26 15:36:54,6006,5679,97,819,5,20.0
3,2016-05-26 16:36:54,6067,5665,18,121,2,3.0
4,2016-05-26 17:06:54,6093,5709,11,2222,5,20.0


<br><br>

<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Feature Selection</h1><br><br>


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">For this exercise, we can choose from buyclicksDF,  the 'price' of each app that a user purchases as an attribute that captures user's purchasing behavior. The following command selects 'userid' and 'price' and drops all other columns that we do not want to use at this stage.</p>


</div>

<br><br><br><br>

In [16]:
userPurchases = buyclicksDF[['userId','price']] #select only userid and price
userPurchases.head(n=5)

,userId,price
0,1300,3.0
1,868,10.0
2,819,20.0
3,121,3.0
4,2222,20.0


<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Similarly, from the adclicksDF,  we will use the 'adCount' as an attribute that captures user's inclination to click on ads. The following command selects 'userid' and 'adCount' and drops all other columns that we do not want to use at this stage.</p>


</div>

<br><br><br><br>

In [17]:
useradClicks = adclicksDF[['userId','adCount']]

In [18]:
useradClicks.head(n=5) #as we saw before, this line displays first five lines

,userId,adCount
0,611,1
1,1874,1
2,2139,1
3,212,1
4,1027,1



<h1 style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">Step 2: Training Data Set Creation</h1>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Create the first aggregate feature for clustering</h1><br><br> 


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">From each of these single ad-clicks per row, we can now generate total ad clicks per user. Let's pick a user with userid = 3. To find out how many ads this user has clicked overall, we have to find each row that contains userid = 3, and report the total number of such rows.
The following commands sum the total number of ads per user and rename the columns to be called 'userid' and 'totalAdClicks'.  Note that you may not need to aggregate (e.g. sum over many rows) if you choose a different feature and your data set already provides the necessary information. In the end, we want to get one row per user, if we are performing clustering over users.

</div>

In [24]:
adsPerUser = useradClicks.groupby('userId').sum()
adsPerUser = adsPerUser.reset_index()
adsPerUser.columns = ['userId', 'totalAdClicks'] #rename the columns



<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Let us display the first 5 lines of 'adsPerUser' to see if there
is a column named 'totalAdClicks' containing total adclicks per user.</p>

</div>


In [25]:
adsPerUser.head(n=5)

,userId,totalAdClicks
0,1,44
1,8,10
2,9,37
3,10,19
4,12,46




<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Create the second aggregate feature for clustering</h1><br><br> 


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Similar to what we did for adclicks, here we find out how much money in total did each user spend on buying in-app purchases. As an example, let's pick a user with userid = 9. To find out the total money spent by this user, we have to find each row that contains userid = 9, and report the sum of the column'price' of each product they purchased.

The following commands sum the total money spent by each user and rename the columns to be called 'userid' and 'revenue'.
<br><br>

<p style="line-height:31px;"> <b> Note: </b> that you can also use other aggregates, such as sum of money spent on a specific ad category by a user or on a set of ad categories by each user, game clicks per hour by each user etc. You are free to use any mathematical operations on the fields provided in the CSV files when creating features. </p>


</div>


In [26]:
revenuePerUser = userPurchases.groupby('userId').sum()
revenuePerUser = revenuePerUser.reset_index()
revenuePerUser.columns = ['userId', 'revenue'] #rename the columns

In [27]:
revenuePerUser.head(n=5)

,userId,revenue
0,1,21.0
1,8,53.0
2,9,80.0
3,10,11.0
4,12,215.0


<br><br>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Merge the two tables</h1><br><br> 

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Lets see what we have so far. We have a table called revenuePerUser, where each row contains total money a user (with that 'userid') has spent. We also have another table called adsPerUser where each row contains total number of ads a user has clicked. We will use revenuePerUser and adsPerUser as features / attributes to capture our users' behavior.

Let us combine these two attributes (features) so that each row contains both attributes per user. Let's merge these two tables to get one single table we can use for K-Means clustering.
</div>

<br><br><br><br> 

In [28]:
combinedDF = adsPerUser.merge(revenuePerUser, on='userId') #userId, adCount, price

In [29]:
combinedDF.head(n=5) #display how the merged table looks

,userId,totalAdClicks,revenue
0,1,44,21.0
1,8,10,53.0
2,9,37,80.0
3,10,19,11.0
4,12,46,215.0


<br><br>

<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Create the final training dataset</h1><br><br> 

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Our training data set is almost ready. At this stage we can remove the 'userid' from each row, since 'userid' is a computer generated random number assigned to each user. It does not capture any behavioral aspect of a user. One way to drop the 'userid', is to select the other two columns. </p>

</div>

<br><br><br><br>

In [30]:
trainingDF = combinedDF[['totalAdClicks','revenue']]
trainingDF.head(n=5)

,totalAdClicks,revenue
0,44,21.0
1,10,53.0
2,37,80.0
3,19,11.0
4,46,215.0


<br><br>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Display the dimensions of the training dataset</h1><br><br> 


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Display the dimension of the training data set. To display the dimensions of the trainingDF, simply add .shape as a suffix and hit enter.</p>

</div>

<br><br><br><br>


In [31]:
trainingDF.shape

(543, 2)

<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">The following two commands convert the tables we created into a format that can be understood by the KMeans.train function. <br><br>

line[0] refers to the first column. line[1] refers to the second column. If you have more than 2 columns in your training table, modify this command by adding line[2], line[3], line[4] ...</p>

</div>

<br><br><br><br>


In [57]:
from pyspark import SparkContext
try:
    SparkContext(app=App_name, master=local[4]).stop()
except:
    pass
sc=SparkContext('local','pyspark')




Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext may be running in this JVM (see SPARK-2243). To ignore this error, set spark.driver.allowMultipleContexts = true. The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

In [45]:
sqlContext = SQLContext(sc)
pDF = sqlContext.createDataFrame(trainingDF)
parsedData = pDF.rdd.map(lambda line: array([line[0], line[1]])) #totalAdClicks, revenue

<br>
<h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Step 3: Train to Create Cluster Centers</h1>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Train KMeans model</h1><br><br> 

<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Here we are creating two clusters as denoted in the second argument.</p>

</div>

<br><br><br><br>


In [47]:
my_kmmodel = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Display the centers of two clusters formed</h1><br><br> 

In [49]:
print(my_kmmodel.centers)

[array([  39.43434343,  111.78787879]), array([ 27.13063063,  23.1981982 ])]


<br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Step 4: Recommend Actions</h1>
<br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Analyze the cluster centers
</h1>

<br><br> 

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Each array denotes the center for a cluster:<br><br>
One Cluster is centered at   ... array([ 29.43211679,  24.21021898])<br>
Other Cluster is centered at   ... array([  42.05442177,  113.02040816])</p>

<br><br>

<p style="line-height:31px;"> First number (field1) in each array refers to number of ad-clicks and the second number (field2) is the revenue per user.

Compare the 1st number of each cluster to see how differently users in each cluster behave when it comes to clicking ads.

Compare the 2nd number of each cluster to see how differently users in each cluster behave when it comes to buying stuff. 

</p><br><br>

<p style="line-height:31px;">In one cluster, in general, players click on ads much more often (~1.4 times) and spend more money (~4.7 times) on in-app purchases. Assuming that Eglence Inc. gets paid for showing ads and for hosting in-app purchase items, we can use this information to increase game's revenue by increasing the prices for ads we show to the frequent-clickers, and charge higher fees for hosting the in-app purchase items shown to the higher revenue generating buyers.</p>

<br><br>
<p style="line-height:31px;"> <b> Note: </b>  This analysis requires you to compare the cluster centers and find any ‘significant’ differences in the corresponding feature values of the  centers. The answer to this question will depend on the features you have chosen. <br><br> Some features help distinguish the clusters remarkably while others may not tell you much. At this point, if you don’t find clear distinguishing patterns, perhaps re-running the clustering model with different numbers of clusters and revising the features you picked would be a good idea. </p>

</div>
